In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
   # for filename in filenames:
      #  print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
import cv2
from tensorflow import keras
from tensorflow.keras import layers, Input
from keras.layers import InputLayer, MaxPooling2D, Flatten, Dense, Conv2D, Dropout
from keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions, ResNet50
from tensorflow.keras.optimizers import Adam, SGD

# لخلط البيانات 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from PIL.Image import open

from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import random
%matplotlib inline

In [3]:
# Constants
#../input/brain-tumor
#../input/brain-tumor/Brain Tumor/Brain Tumor
IMAGE_DATASET = "../input/brain-tumor/Brain Tumor/Brain Tumor"
IMAGE_DATASET_RAW = r"..\input\brain-tumor\Brain Tumor\Brain Tumor"
WORKING_FOLDER = "/kaggle/working"
# الابعاد مع عدد المحاولات 
IMG_HEIGHT = 224
IMG_WIDTH = 224
EPOCHS = 50

In [4]:
cortex_df = pd.read_csv("../input/brain-tumor/Brain Tumor.csv")
plt.figure(figsize=(20,20))
test_folder="../input/brain-tumor/Brain Tumor/Brain Tumor" 
for i in range(5):
    file = random.choice(os.listdir(test_folder))
    image_path= os.path.join(test_folder, file)
    img=mpimg.imread(image_path)
    ax=plt.subplot(1,5,i+1)
    ax.title.set_text(file)
    plt.imshow(img)
    

# **Loading Dataset**

In [21]:
dataset_df = pd.DataFrame()
dataset_df["Image"] = cortex_df["Image"]
dataset_df["Class"] = cortex_df["Class"]
path_list = []
for img_path in os.listdir(IMAGE_DATASET):
    path_list.append( os.path.join(IMAGE_DATASET,img_path))
path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in path_list}
dataset_df["paths"] = cortex_df["Image"].map(path_dict.get)
dataset_df["pixels"] = dataset_df["paths"].map(lambda x:np.asarray(open(x).resize((IMG_HEIGHT,IMG_WIDTH))))
dataset_df.head(10)

In [24]:
image_list = []
for i in range(len(dataset_df)):
    brain_image = dataset_df["pixels"][i].astype(np.float32)
    brain_image /= 255
    image_list.append(brain_image)
X = np.array(image_list)
print(X.shape)

In [25]:
y = np.array(dataset_df.Class)
y.shape

# **Spliting Dataset**

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print('The shape of the X_train :'+' '+str(X_train.shape))
print('The size of the X_train :'+' '+str(y_train.shape))
print('The shape of the X_test :'+' '+str(X_test.shape))
print('The size of the X_test:'+' '+str(y_test.shape))

In [27]:
from tensorflow.keras.layers import AveragePooling2D
loss_fn = BinaryCrossentropy(from_logits=True)

# **Implementing Model**

In [28]:
model=Sequential()
model.add(Conv2D(16,(3,3),padding="valid",strides=2, activation="relu",input_shape=(224,224,3)))
model.add(Conv2D(16,(5,5),padding="valid",strides=2, activation="relu"))
model.add(AveragePooling2D(pool_size=(3,3),strides=2,padding="same"))
model.add(Conv2D(32,(3,3),padding="same",strides=1,activation='relu'))
model.add(AveragePooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3),padding="valid",strides=1,activation='relu'))
model.add(Conv2D(64, kernel_size=3, strides=(2, 2), padding="same", activation="relu"))
model.add(Conv2D(64, kernel_size=3, strides=(2, 2), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last", padding='same'))
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(120,activation="relu"))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss=loss_fn, optimizer='adam', metrics=['accuracy'] )


In [29]:
model.summary()

# **Training**

In [30]:
print(X_train.shape)
print(y_train.shape)
from time import time
tic= time()
epoch = 15
hist=model.fit(X_train, y_train, validation_split= 0.2, epochs=epoch, batch_size=100, verbose=1)
toc=time()
print("Training Time : {}".format(toc-tic))

In [31]:
def decode_sentiment(score):
    return 1 if score>0.5 else 0

# **Model Evaluation**

In [32]:
from sklearn import metrics
scores = model.predict(X_test)
y_pred = [decode_sentiment(score) for score in scores]
metrics.accuracy_score(y_test, y_pred)


In [33]:
print("classification report:")
print(metrics.classification_report(y_test, y_pred))#, target_names=target_names))

In [34]:
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.plot(hist.history['loss'], label='Loss')
plt.plot(hist.history['val_loss'], label='val_Loss')
plt.legend()
plt.grid()
plt.title('Loss evolution')

In [35]:
plt.subplot(1, 1, 1)
plt.plot(hist.history['accuracy'], label='accuracy')
plt.plot(hist.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.grid()
plt.title('Accuracy evolution')

In [36]:

import seaborn as sns
print("confusion matrix: ")
cf_matrix=metrics.confusion_matrix(y_test, y_pred)
sns.heatmap(cf_matrix, annot=True)

In [37]:
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues')